# Named Entity Recognition in Nepali Language

This google colab is about named entity recognition in cyber security domain by fine-tuning BERT-derivative model with the [CyNER](https://github.com/aiforsec/CyNER) dataset. This dataset is related to the vulnerabilities, firmware and cyber security.

## Installation

In [1]:
!python3 -m pip install -U huggingface_hub
!python3 -m pip install -U transformers
!python3 -m pip install -U datasets evaluate
!python3 -m pip install -U accelerate
!python3 -m pip install -U seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=db1fee58b916805aafc61288539e2f975039828423952430a10be338a4ba74d1
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
# Wrap the text in ipython notebook
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Data Preprocessing

## Load NepSA dataset


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
filepath = '/content/drive/MyDrive/nepsa_data/'

In [5]:
from datasets import load_dataset

data_files = {
    "train": filepath + "train.txt",
    "validation": filepath + "valid.txt",
    "test": filepath + "test.txt",
}

raw_datasets = load_dataset(filepath + "load_ner.py", data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Check the basic information on the loaded dataset

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2323
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 330
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 280
    })
})

Check sample of tokens from train dataset

In [7]:
print(raw_datasets["train"][0]["tokens"])

['भालुनी', 'सावित्री', 'कुकुरनी', 'मिले', 'को', 'रहेछ', 'आजा', 'प्रक्षया', 'थाहा', 'भयो', 'निर्माल', 'बहिनी', 'लाई', 'बलत्कार', 'गर्न', 'लगाउने', 'यनि', 'भलु', 'हरु', 'रहेछ', 'पहिला', 'जाती', 'आन्दोलान', 'गरे', 'को', 'थियो', 'त्यो', 'सबै', 'यनि', 'हरु', 'को', 'नाटक', 'रहेछ', 'आजा', 'बल', 'थाहा', 'भयो', '।']


Check the NER tags (its IDS) of the corresponding sample

In [8]:
print(raw_datasets["train"][0]["ner_tags"])

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PROFANITY', 'I-PROFANITY', 'B-FEEDBACK', 'I-FEEDBACK', 'B-GENERAL', 'I-GENERAL', 'B-VIOLENCE', 'I-VIOLENCE'], id=None), length=-1, id=None)

Check the labels in the dataset

In [10]:
label_names = ner_feature.feature.names
label_names # GENERAL = slightly negative or positive connotation giving word

['O',
 'B-PROFANITY',
 'I-PROFANITY',
 'B-FEEDBACK',
 'I-FEEDBACK',
 'B-GENERAL',
 'I-GENERAL',
 'B-VIOLENCE',
 'I-VIOLENCE']

Display the token and labels

In [11]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

for x, y in zip(line1.split(), line2.split()):
    print(x, '\t', y)

भालुनी 	 B-PROFANITY
सावित्री 	 O
कुकुरनी 	 B-PROFANITY
मिले 	 O
को 	 O
रहेछ 	 O
आजा 	 O
प्रक्षया 	 O
थाहा 	 O
भयो 	 O
निर्माल 	 O
बहिनी 	 O
लाई 	 O
बलत्कार 	 B-VIOLENCE
गर्न 	 O
लगाउने 	 O
यनि 	 O
भलु 	 B-PROFANITY
हरु 	 O
रहेछ 	 O
पहिला 	 O
जाती 	 O
आन्दोलान 	 O
गरे 	 O
को 	 O
थियो 	 O
त्यो 	 O
सबै 	 O
यनि 	 O
हरु 	 O
को 	 O
नाटक 	 O
रहेछ 	 O
आजा 	 O
बल 	 O
थाहा 	 O
भयो 	 O
। 	 O


## Tokenization

In [95]:
from transformers import AutoTokenizer

# model_checkpoint = "NepBERTa/NepBERTa"
# model_checkpoint = "Rajan/NepaliBERT"
# model_checkpoint = "Rajan/nepbertaTorch"
# model_checkpoint = "Sakonii/distilbert-base-nepali"
# model_checkpoint = "Sakonii/deberta-base-nepali"
model_checkpoint = "mrm8488/bert-multi-cased-finetuned-xquadv1"
# model_checkpoint = "xlm-roberta-large"
# model_checkpoint = "nlptown/bert-base-multilingual-uncased-sentiment"
# model_checkpoint = "bert-base-multilingual-uncased"
# model_checkpoint = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [96]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', 'भ', '##ाल', '##ुन', '##ी', 'स', '##ाव', '##ित्र', '##ी', 'क', '##ुक', '##ुर', '##नी', 'म', '##िले', 'को', 'रहे', '##छ', 'आज', '##ा', 'प', '##्र', '##क्ष', '##या', 'था', '##हा', 'भयो', 'न', '##िर', '##्मा', '##ल', 'ब', '##ह', '##िनी', 'ला', '##ई', 'ब', '##ल', '##त', '##्कार', 'गर्न', 'लगा', '##उने', 'य', '##नि', 'भ', '##ल', '##ु', 'हर', '##ु', 'रहे', '##छ', 'प', '##ह', '##िला', 'जाती', 'आ', '##न्द', '##ोल', '##ान', 'गरे', 'को', 'थियो', 'त', '##्यो', 'सबै', 'य', '##नि', 'हर', '##ु', 'को', 'नाटक', 'रहे', '##छ', 'आज', '##ा', 'ब', '##ल', 'था', '##हा', 'भयो', '।', '[SEP]']


In [97]:
print(inputs.word_ids())

[None, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, 7, 8, 8, 9, 10, 10, 10, 10, 11, 11, 11, 12, 12, 13, 13, 13, 13, 14, 15, 15, 16, 16, 17, 17, 17, 18, 18, 19, 19, 20, 20, 20, 21, 22, 22, 22, 22, 23, 24, 25, 26, 26, 27, 28, 28, 29, 29, 30, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 37, None]


## Data Preprocessing

In [98]:
# Align the number of labels and the tokens
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [99]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 1, 2, 2, 2, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [100]:
# Helper function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [101]:
# Tokenize all the examples from the datasets
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/2323 [00:00<?, ? examples/s]

Map:   0%|          | 0/330 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

# Fine Tuning

## Data Collation

Prepare the dataloader for the training session

In [102]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    1,    2,    2,    2,    0,    0,    0,    0,    1,    2,    2,
            2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            7,    8,    8,    8,    0,    0,    0,    0,    0,    1,    2,    2,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    5,    6,    6,    6,    6,    6,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, 

In [103]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 1, 2, 2, 2, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


## Setup Evaluation

In [104]:
import evaluate
import numpy as np

metric = evaluate.load("seqeval")


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [105]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [106]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    # from_tf=True,
)


model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at mrm8488/bert-multi-cased-finetuned-xquadv1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [107]:
model.config.num_labels

9

## Training

In [108]:
from google.colab import userdata
from huggingface_hub import login, notebook_login

# notebook_login()
login(token=userdata.get('hugging_face'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Optimization

In [109]:
# from torch import nn
# from transformers import Trainer


# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         # forward pass
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
#         # compute custom loss (suppose one has 3 labels with different weights)
#         loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0], device=model.device))
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

In [110]:
# from transformers.optimization import Adafactor, AdafactorSchedule

# optimizer = Adafactor(
#     model.parameters(),
#     lr=1e-3,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False,
# )

# lr_scheduler = AdafactorSchedule(optimizer)
# trainer = Trainer(..., optimizers=(optimizer, lr_scheduler))

In [111]:
from transformers import TrainingArguments, Trainer

model_name = "nepsa-ner"

args = TrainingArguments(
    model_name,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.550839,0.160083,0.165236,0.162619,0.810709
2,0.582400,0.492863,0.217042,0.289700,0.248162,0.829104
3,0.582400,0.501856,0.227554,0.315451,0.264388,0.823684


TrainOutput(global_step=873, training_loss=0.4931761227112865, metrics={'train_runtime': 174.841, 'train_samples_per_second': 39.859, 'train_steps_per_second': 4.993, 'total_flos': 254463516535554.0, 'train_loss': 0.4931761227112865, 'epoch': 3.0})

In [119]:
trainer.evaluate()

{'eval_loss': 0.5018563270568848,
 'eval_precision': 0.22755417956656346,
 'eval_recall': 0.315450643776824,
 'eval_f1': 0.26438848920863306,
 'eval_accuracy': 0.8236839944157017,
 'eval_runtime': 2.1076,
 'eval_samples_per_second': 156.574,
 'eval_steps_per_second': 19.928,
 'epoch': 3.0}

## Save the model

In [120]:
saved_model_path='nepsa_ner-1/'
trainer.save_model(saved_model_path)

## Evaluation

In [121]:
predictions = trainer.predict(tokenized_datasets["test"])

In [122]:
from tabulate import tabulate

metrics = ['precision', 'recall', 'f1', 'accuracy']
prediction_results = []

for key, val in predictions.metrics.items():
    if any(item in key for item in metrics):
        prediction_results.append([key, str(round(val,4)*100)+'%'])

print(tabulate(prediction_results, headers=['Metric', 'Score']))

Metric          Score
--------------  -------------------
test_precision  25.319999999999997%
test_recall     33.01%
test_f1         28.660000000000004%
test_accuracy   83.67%


## Inference

In [123]:
from transformers import pipeline

token_classifier = pipeline(
    "token-classification", model=saved_model_path, aggregation_strategy="simple"
)


In [124]:
results = token_classifier("वा जैनु जि तपाइलाइ धन्यबाद छ गगन्या चोर लाई मुख भरी जबाब दिएको मा")
#ओली दलाल मुर्दाबाद
#यो गोविन्दे लाई देश निकाला गर्नु पर्छ")
#यो मुला गोबिन्द ठिक छैन
#यो गोविन्दे लाई देश निकाला गर्नु पर्छ"
 #यो पुण्य गौतम जड्या हो जस्तो कस कस लाई लाग्छ ।")

In [125]:
prediction_results = []
for each_entity in results:
    prediction_results.append([each_entity['word'], each_entity['entity_group']])

print(tabulate(prediction_results, headers=['Word', 'Predictions']))


Word    Predictions
------  -------------
धन्यबाद   GENERAL
चोर      PROFANITY
